In [1]:
from pathlib import Path

import numpy as np

import FewShotCVRP.ea as ea
import FewShotCVRP.ea_parallel as ea_parallel
from FewShotCVRP.dataset.theta_control_loader import DescriptorRepository
from FewShotCVRP.examples.params_search import simulation


In [2]:
descriptor_path = Path("../../dataset/trained_models/per_instance/descriptor.json")
repo = DescriptorRepository(descriptor_path)

instance = next(inst for inst in repo.instances if inst.name == "X-n209-k16")
cvrp = simulation.get_cvrp_instance("X-n209-k16.xml")

cfg = repo.ea_config
lambda_ = int(cfg["ea_lambda_"])
generations_number = int(cfg["ea_generations_number"])
max_evals = generations_number * lambda_
seed = 2024


In [27]:
wrapper_controller = repo.create_theta_controller(instance)
baseline_controller = repo.create_theta_controller(instance)

theta_schedule_window = np.asarray(baseline_controller.theta_schedule_window(), dtype=float)
theta_control_fun = baseline_controller._theta_function


In [ ]:
res_wrapper = ea_parallel.one_plus_lambda_ea_parallel_nn_control(
    cvrp,
    wrapper_controller,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation="2opt",
    numproc=1,
    verbose=False,
)

res_original = ea.one_plus_lambda_ea_with_theta_control(
    cvrp,
    theta_control_fun=theta_control_fun,
    window=len(theta_schedule_window),
    theta_schedule_window=theta_schedule_window,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation=ea.mutate_shift_2opt_fast_fast,
    verbose=False,
)


In [5]:
comparison = {
    "best_fitness_equal": np.isclose(res_wrapper["best_fitness"], res_original["best_fitness"]),
    "evals_equal": res_wrapper["evals"] == res_original["evals"],
    "gens_equal": res_wrapper["gens"] == res_original["gens"],
    "history_equal": res_wrapper["history"] == res_original["history"],
}
comparison


{'best_fitness_equal': np.True_,
 'evals_equal': True,
 'gens_equal': True,
 'history_equal': True}

In [45]:
from importlib import reload

reload(ea_parallel)

res_wrapper = ea_parallel.one_plus_lambda_ea_parallel_nn_control(
    cvrp,
    wrapper_controller,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation="2opt",
    numproc=50,
    verbose=False,
)
res_wrapper["best_fitness"]

52417.0

### Run ea_voting

In [22]:
from importlib import reload

reload(ea)

cvrp = simulation.get_cvrp_instance("X-n209-k16.xml")
res_voting = ea.one_plus_lambda_ea_voting(
    cvrp,
    theta=100.0,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=2024,
    mutation=ea.mutate_shift_2opt_fast_fast,
    verbose=True,
    scale_quorum=True,
)
res_voting["best_fitness"]


Quorum: 2591.9999991900404
[init] fitness=118362.000000
[gen 1] theta=100 v=3520 cnt=6337 fitness=107301.000000 ↑ evals=20001/1000001
[gen 2] theta=100 v=3717 cnt=6537 fitness=102748.000000 ↑ evals=40001/1000001
[gen 3] theta=50 v=3826 cnt=6646 fitness=98195.000000 ↑ evals=60001/1000001
[gen 4] theta=25 v=4121 cnt=6943 fitness=95218.000000 ↑ evals=80001/1000001
[gen 5] theta=12.5 v=4838 cnt=7689 fitness=92321.000000 ↑ evals=100001/1000001
[gen 6] theta=7 v=987 cnt=1030 fitness=90468.000000 ↑ evals=120001/1000001
[gen 7] theta=7 v=1634 cnt=1698 fitness=88644.000000 ↑ evals=140001/1000001
[gen 8] theta=14 v=1971 cnt=2036 fitness=86798.000000 ↑ evals=160001/1000001
[gen 9] theta=7 v=3304 cnt=3427 fitness=84541.000000 ↑ evals=180001/1000001
[gen 10] theta=4.9 v=963 cnt=1024 fitness=82801.000000 ↑ evals=200001/1000001
[gen 11] theta=9.8 v=1216 cnt=1277 fitness=81613.000000 ↑ evals=220001/1000001
[gen 12] theta=4.9 v=1938 cnt=2026 fitness=80200.000000 ↑ evals=240001/1000001
[gen 13] theta=4.

54090.0

In [31]:
from importlib import reload

reload(ea)

cvrp = simulation.get_cvrp_instance("X-n641-k35.xml")
res_voting = ea.one_plus_lambda_ea_with_one_fifth_success(
    cvrp,
    theta=100.0,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=2024,
    mutation=ea.mutate_shift_2opt_fast_fast,
    verbose=True
)
res_voting["best_fitness"]


[init] fitness=337473.000000
[gen 1] theta=100 fitness=323009.000000 ↑ evals=20001/1000001
[gen 2] theta=100 fitness=312327.000000 ↑ evals=40001/1000001
[gen 3] theta=66.6667 fitness=306448.000000 ↑ evals=60001/1000001
[gen 4] theta=44.4444 fitness=300826.000000 ↑ evals=80001/1000001
[gen 5] theta=29.6296 fitness=296373.000000 ↑ evals=100001/1000001
[gen 6] theta=19.7531 fitness=291940.000000 ↑ evals=120001/1000001
[gen 7] theta=13.1687 fitness=288449.000000 ↑ evals=140001/1000001
[gen 8] theta=8.77915 fitness=284713.000000 ↑ evals=160001/1000001
[gen 9] theta=5.85277 fitness=280581.000000 ↑ evals=180001/1000001
[gen 10] theta=3.90184 fitness=277680.000000 ↑ evals=200001/1000001
[gen 11] theta=2.60123 fitness=275146.000000 ↑ evals=220001/1000001
[gen 12] theta=1.73415 fitness=273242.000000 ↑ evals=240001/1000001
[gen 13] theta=1.1561 fitness=271187.000000 ↑ evals=260001/1000001
[gen 14] theta=1 fitness=269355.000000 ↑ evals=280001/1000001
[gen 15] theta=1.5 fitness=267077.000000 ↑ eval

216129.0

In [29]:
res_wrapper = ea_parallel.one_plus_lambda_ea_parallel_nn_control(
    cvrp,
    wrapper_controller,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation="2opt",
    numproc=1,
    verbose=True,
)

[init] fitness=118362.000000
[gen 1] theta=100 fitness=107401.000000 ↑ evals=20001/1000001
[gen 2] theta=100 fitness=102458.000000 ↑ evals=40001/1000001
[gen 3] theta=50 fitness=99326.000000 ↑ evals=60001/1000001
[gen 4] theta=50 fitness=96273.000000 ↑ evals=80001/1000001
[gen 5] theta=20 fitness=93968.000000 ↑ evals=100001/1000001
[gen 6] theta=3.95 fitness=91712.000000 ↑ evals=120001/1000001
[gen 7] theta=7.58 fitness=89808.000000 ↑ evals=140001/1000001
[gen 8] theta=3.42 fitness=87993.000000 ↑ evals=160001/1000001
[gen 9] theta=3.93 fitness=86173.000000 ↑ evals=180001/1000001
[gen 10] theta=3.53 fitness=84752.000000 ↑ evals=200001/1000001
[gen 11] theta=2.7 fitness=83369.000000 ↑ evals=220001/1000001
[gen 12] theta=2.78 fitness=82221.000000 ↑ evals=240001/1000001
[gen 13] theta=2.71 fitness=81102.000000 ↑ evals=260001/1000001
[gen 14] theta=2.51 fitness=79604.000000 ↑ evals=280001/1000001
[gen 15] theta=2.6 fitness=78599.000000 ↑ evals=300001/1000001
[gen 16] theta=2.5 fitness=77486